<a href="https://colab.research.google.com/github/EdWangLoDaSc/Dropout-as-a-Grid-Search_Representing-Model-Uncertainty-in-Deep-Learning/blob/main/rmb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install tensorflow==1.14.0
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from google.colab import drive
drive.mount('./content')


ValueError: ignored

In [46]:
import os
import sys
import pandas as pd
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.decomposition import PCA
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision.datasets import MNIST
module_path = os.path.abspath('/content/content/MyDrive/pytorch-rbm-autoencoder-main')
os.chdir(module_path)

OSError: ignored

In [ ]:
from dae import DAE
from demo_train_utils import train_rbm
from rbm import RBM
from utils import *


In [ ]:
print(f"Using device {DEVICE}")
seed_everything(42)



In [44]:
hidden_dimensions = [
    {
        "hidden_dim": 700, 
        "num_epochs": 100, 
        "learning_rate": 0.001, 
        "display_dim1": 28, 
        "display_dim2": 28, 
        "use_gaussian": False
    }, 
    {
        "hidden_dim": 256, 
        "num_epochs": 100, 
        "learning_rate": 0.001, 
        "use_gaussian": False
    },
    {
        "hidden_dim": 128, 
        "num_epochs": 100, 
        "learning_rate": 0.001, 
        "use_gaussian": False
    },
    {
        "hidden_dim": 64, 
        "num_epochs": 100, 
        "learning_rate": 0.001, 
        "use_gaussian": False
    },
    {
        "hidden_dim": 32, 
        "num_epochs": 100, 
        "learning_rate": 0.001, # use much lower LR for gaussian to avoid exploding gradient
        "use_gaussian": True # use a Gaussian distribution for the last hidden layer to let it take advantage of continuous values
    }
]


In [45]:
datasets = pd.read_csv('/content/content/MyDrive/pytorch-rbm-autoencoder-main/FP_3_512_D1.csv').values
datasets.shape
datasets = DataLoader(
    TensorDataset(torch.Tensor(datasets).to(DEVICE)),
    batch_size=16,
    shuffle=False
)

OSError: ignored

In [39]:
new_train_dl = datasets

visible_dim = 512
hidden_dim = None
models = [] # trained RBM models
for configs in hidden_dimensions:
    
    # parse configs
    hidden_dim = configs["hidden_dim"]
    num_epochs = configs["num_epochs"]
    lr = configs["learning_rate"]
    use_gaussian = configs["use_gaussian"]
    
    # train RBM
    print(50 * '-')
    print(f"{visible_dim} to {hidden_dim}")
    model, v, v_pred = train_rbm(new_train_dl, visible_dim, hidden_dim, k=1, num_epochs=num_epochs, lr=lr, use_gaussian=use_gaussian)
    models.append(model)
    
    # display sample output
    #display_output(v, v_pred, d1, d2)

    # rederive new data loader based on hidden activations of trained model
    new_data = []
    for data_list in new_train_dl:
        p = model.sample_h(data_list[0])[0]
        new_data.append(p.detach().cpu().numpy())
    new_input = np.concatenate(new_data)
    new_train_dl = DataLoader(
        TensorDataset(torch.Tensor(new_input).to(DEVICE)), 
        batch_size=16, 
        shuffle=False
    )
    
    # update new visible_dim for next RBM
    visible_dim = hidden_dim


--------------------------------------------------
512 to 700
epoch 0: 0.07056031376123428
epoch 1: 0.05753840506076813
epoch 2: 0.052218448370695114
epoch 3: 0.04920882731676102
epoch 4: 0.0472663976252079
epoch 5: 0.04182451218366623
epoch 6: 0.03317488357424736
epoch 7: 0.02865113504230976
epoch 8: 0.025878526270389557
epoch 9: 0.024073343724012375
epoch 10: 0.02270248904824257
epoch 11: 0.021667564287781715
epoch 12: 0.020888157188892365
epoch 13: 0.020286867395043373
epoch 14: 0.019751518964767456
epoch 15: 0.019359983503818512
epoch 16: 0.018938235938549042
epoch 17: 0.018597867339849472
epoch 18: 0.018412141129374504
epoch 19: 0.01813599281013012
epoch 20: 0.017971279099583626
epoch 21: 0.017762424424290657
epoch 22: 0.017598658800125122
epoch 23: 0.017433203756809235
epoch 24: 0.01733900047838688
epoch 25: 0.01721319556236267
epoch 26: 0.017123151570558548
epoch 27: 0.016993381083011627
epoch 28: 0.0169048011302948
epoch 29: 0.01680287905037403
epoch 30: 0.016706272959709167
ep